In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from os.path import join
from collections import defaultdict
import matplotlib.pyplot as plt
import brainbox.io.one as bbone
from reproducible_ephys_functions import query
from tqdm import notebook
import brainbox as bb
from ibllib.io import spikeglx
import brainbox.behavior.wheel as wh
from brainbox import singlecell
from brainbox.metrics.single_units import spike_sorting_metrics
from one.api import ONE
import alf

from ibllib.qc.camera import CameraQC
import os
from scipy.stats import zscore
from collections import Counter

from utils import *

one = ONE()

In [ ]:
mtnn_criteria = check_mtnn_criteria()

In [ ]:
# load all rs trajectories
trajs = query()

In [ ]:
for eid in mtnn_criteria.keys():
    print(eid)
    print(mtnn_criteria[eid])
    for traj in trajs:
        if traj['session']['id'] == eid:
            print(traj)
    print('------------------------------------')

In [2]:
mtnn_eids = get_mtnn_eids()
print(mtnn_eids)

{'56b57c38-2699-4091-90a8-aba35103155e': 0}


In [3]:
# run Charles's behavioral model
# run_exp_prevAction(mtnn_eids)

In [4]:
traj = get_traj(mtnn_eids)
print(traj)

[{'id': '0e6c55bc-d2af-45e1-b960-87b7d203dc89', 'probe_insertion': 'ce397420-3cd2-4a55-8fd1-5e28321981f4', 'x': -2243.0, 'y': -2000.0, 'z': -169.0, 'depth': 4000.0, 'theta': 15.0, 'phi': 180.0, 'roll': 0.0, 'provenance': 'Planned', 'session': {'subject': 'SWC_054', 'start_time': '2020-10-05T19:15:05.974514', 'number': 1, 'lab': 'mrsicflogellab', 'id': '56b57c38-2699-4091-90a8-aba35103155e', 'task_protocol': '_iblrig_tasks_ephysChoiceWorld6.4.2'}, 'probe_name': 'probe01', 'coordinate_system': None, 'datetime': '2020-12-05T15:34:21.905537', 'json': None}]


In [5]:
# ts_directory = os.path.join('.', "new_timestamps")
# if not os.path.exists(ts_directory):
#     os.makedirs(ts_directory)
# for i in notebook.tqdm(range(len(traj))):
#     eid = traj[i]['session']['id']
#     get_new_timestamps(eid, 'left', ts_directory)

In [6]:
feature_list = []
output_list = []
trial_length_array_list = []
cluster_number_list = []
session_list = []
nan_trials = []
session_count = {'mainenlab': 0, 'churchlandlab': 0, ('hoferlab', 'mrsicflogellab'): 0, 'danlab': 0}
for i in notebook.tqdm(range(len(traj))):
    feature, output, trial_length_array, cluster_numbers, nan_idx, success = featurize(i, 
                                                                                       traj[i], 
                                                                                       one, 
                                                                                       session_count)
    if not success:
        print('not successful')
        continue
    else:
        print('successful')
    feature_list.append(feature)
    output_list.append(output)
    trial_length_array_list.append(trial_length_array)
    cluster_number_list.append(cluster_numbers)
    session_list.append(traj[i])
    nan_trials.append(nan_idx)

  0%|          | 0/1 [00:00<?, ?it/s]

2022-02-28 02:34:15.806 WARNING  [one.py:515] Deprecation warning: brainbox.io.one.load_spike_sorting will be removed in future versions.Use brainbox.io.one.SpikeSortingLoader instead


processing SWC_054: 56b57c38-2699-4091-90a8-aba35103155e


2022-02-28 02:34:16.718 WARNING  [one.py:462] Deprecation warning: brainbox.io.one.load_spike_sorting will be removed in future versions.Use brainbox.io.one.SpikeSortingLoader instead


spikes retrieved
loading motion energy and dlc
motion energy + dlc retrieved
loading pLeft
trial info retrieved
passive data loaded
number of trials found: 697 (active: 517, passive: 180)
getting lick times
getting paw speed
getting nose speed
getting pupil diameter
getting wheel velocity
all brain region counts:  Counter({'PPC': 55, 'PO': 49, 'LP': 49, 'CA1': 12, 'DG': 3})
good cluster id:  [109 120 166 115 165 162 364 309 122 289]
selected brain region counts:  Counter({'PO': 7, 'LP': 3})


0it [00:00, ?it/s]

  0%|          | 0/517 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

successful


In [ ]:
for i in range(len(feature_list)):
    print(feature_list[i].shape)

In [ ]:
for i in range(len(output_list)):
    print(output_list[i].shape)

In [ ]:
topk_list = []
for i in range(len(output_list)):
    mean_fr = output_list[i].mean(2).mean(0)
    top_k = mean_fr.argsort()[::-1][:250]
    topk_list.append(top_k)

In [ ]:
truncated_feature_list = []
for i in range(len(feature_list)):
    top_k = topk_list[i]
    feature = feature_list[i][:,top_k]
    for k in range(10):
        feature[k,:,:,0] = 10*i + k
    truncated_feature_list.append(feature)

In [ ]:
truncated_feature = np.asarray(truncated_feature_list)
print(truncated_feature.shape)

In [ ]:
np.save('truncated_feature_original.npy', truncated_feature)

In [ ]:
seqlen_list = []
for i in range(len(trial_length_array_list)):
    top_k = topk_list[i]
    seqlen_list.append(trial_length_array_list[i][top_k])

In [ ]:
seqlen_array = np.asarray(seqlen_list)
print(seqlen_array.shape)

In [ ]:
np.save('seqlen.npy', seqlen_array)

In [ ]:
# normalize truncated_feature: dlc features + xyz location
# first normalize xyz
x_max = truncated_feature[:,:,:,:,xyz_offset].max()
x_min = truncated_feature[:,:,:,:,xyz_offset].min()
y_max = truncated_feature[:,:,:,:,xyz_offset+1].max()
y_min = truncated_feature[:,:,:,:,xyz_offset+1].min()
z_max = truncated_feature[:,:,:,:,xyz_offset+2].max()
z_min = truncated_feature[:,:,:,:,xyz_offset+2].min()

truncated_feature[:,:,:,:,xyz_offset] = 0.1 + 0.9*(truncated_feature[:,:,:,:,xyz_offset] - x_min) / (x_max - x_min)
truncated_feature[:,:,:,:,xyz_offset+1] = 0.1 + 0.9*(truncated_feature[:,:,:,:,xyz_offset+1] - y_min) / (y_max - y_min)
truncated_feature[:,:,:,:,xyz_offset+2] = 0.1 + 0.9*(truncated_feature[:,:,:,:,xyz_offset+2] - z_min) / (z_max - z_min)

# next normalize dlc features
for i in range(stimOnOff_offset - left_dlc_offset):
    idx = left_dlc_offset+i
    
    feature_min = truncated_feature[:,:,:,:,idx].min()
    feature_max = truncated_feature[:,:,:,:,idx].max()
    
    truncated_feature[:,:,:,:,idx] = -1 + 2*(truncated_feature[:,:,:,:,idx] - feature_min) / (feature_max - feature_min)
    
# next normalize wheel
wheel_min = truncated_feature[:,:,:,:,wheel_offset].min()
wheel_max = truncated_feature[:,:,:,:,wheel_offset].max()

truncated_feature[:,:,:,:,wheel_offset] = -1 + 2*(truncated_feature[:,:,:,:,wheel_offset] - wheel_min) / (wheel_max - wheel_min)


# next normalize lick
lick_min = truncated_feature[:,:,:,:,lick_offset].min()
lick_max = truncated_feature[:,:,:,:,lick_offset].max()

truncated_feature[:,:,:,:,lick_offset] = (truncated_feature[:,:,:,:,lick_offset] - lick_min) / (lick_max - lick_min)

# next normalize max_ptp
max_ptp_min = truncated_feature[:,:,:,:,max_ptp_offset].min()
max_ptp_max = truncated_feature[:,:,:,:,max_ptp_offset].max()

truncated_feature[:,:,:,:,max_ptp_offset] = 0.1 + 0.9*(truncated_feature[:,:,:,:,max_ptp_offset] - max_ptp_min) / (max_ptp_max - max_ptp_min)

# next normalize wf_width
wf_width_min = truncated_feature[:,:,:,:,wf_width_offset].min()
wf_width_max = truncated_feature[:,:,:,:,wf_width_offset].max()

truncated_feature[:,:,:,:,wf_width_offset] = 0.1 + 0.9*(truncated_feature[:,:,:,:,wf_width_offset] - wf_width_min) / (wf_width_max - wf_width_min)

In [ ]:
np.save('truncated_feature_normalized.npy', truncated_feature)

In [ ]:
truncated_target_list = []
for i in range(len(output_list)):
    top_k = topk_list[i]
    output = output_list[i][:,top_k]
    truncated_target_list.append(output)

In [ ]:
truncated_target = np.asarray(truncated_target_list)
print(truncated_target.shape)

In [ ]:
np.save('truncated_output.npy', truncated_target)